# Assessment: Unit 6
--------------------

Complete the problems below in your copy of the Jupyter Notebook.

## Problem 6.1.

Load a set of x, y data

Perform linear regression at orders 1, 2, 3 using scikit-learn

Which one has the lowest R2?

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression



## Problem 6.2.

Load data for a 1/x model

create an array of 1/x

run regression

## Problem 6.3.

Load data and plot

What model seems good? 

Run model and test

--------------
## Next Steps:

1. Advance to [Unit 7](../07-advanced-plotting/unit07-lesson.ipynb) when you're ready for the next step